<a href="https://colab.research.google.com/github/qwasd34/DEV_DATA/blob/main/IMDB%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EB%B6%84%EB%A5%98%EA%B8%B0fine_tuning%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 데이터셋과 모델

- IMDB 리뷰 감정 분리

https://huggingface.co/datasets/imdb


https://huggingface.co/google-bert/bert-base-uncased

In [1]:
!pip install datasets
!pip install -U accelerate
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [2]:
from datasets import load_dataset

dataset=load_dataset("imdb")

#훈련세트에서 무작위 500개 샘플 선택
train_sample=dataset["train"].shuffle(seed=42).select(range(500))

#테스트세트에서 무작위 500개 샘플 선택
test_sample=dataset["test"].shuffle(seed=42).select(range(500))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
train_sample, test_sample

(Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }))

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

#사전 훈련된 모델자동로드:
#AutoModelForSequenceClassification 은 from_pretrained 메소드사용
# 해당 모델에 맞는 모델 아키텍처와 가중치 로드
# 분류를 위한 추가 레이어가 생성됨 finetuning 용, 이레이어는 모델의 출력을 최종 분류 결정으로 변환하는 역할 수행
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


# GPU사용 설정 (가능한경우)
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
# 모든 입력크기를 64개의 토큰을 맞춤 길면짤라 짧으면 패딩 수행
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True, padding=True, max_length=64)

encoded_train_sample=train_sample.map(preprocess_function, batched=True)
encoded_test_sample=test_sample.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:
from transformers import Trainer, TrainingArguments

training_args=TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

In [9]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_sample,
    eval_dataset=encoded_test_sample
)

In [11]:
trainer.train()

Step,Training Loss
10,0.720400


Step,Training Loss
10,0.720400
20,0.719700
30,0.709900
40,0.706600
50,0.671700
60,0.644000
70,0.638600
80,0.592200


TrainOutput(global_step=80, training_loss=0.6753782987594604, metrics={'train_runtime': 1579.0963, 'train_samples_per_second': 1.583, 'train_steps_per_second': 0.051, 'total_flos': 82222204800000.0, 'train_loss': 0.6753782987594604, 'epoch': 5.0})

In [12]:
trainer.evaluate()

{'eval_loss': 0.6311379671096802,
 'eval_runtime': 113.6039,
 'eval_samples_per_second': 4.401,
 'eval_steps_per_second': 0.141,
 'epoch': 5.0}

In [13]:
test_result=trainer.predict(test_dataset=encoded_test_sample)

In [14]:
import numpy as np

#예측결과분서:  predict 메소드는 predictions, label_ids, metrics를 포함한 NamedTuple반환
predictions=test_result.predictions

#여기서 prediction배열에는 각테스트 샘플에 대한 예측 결과 저장
predictions=np.argmax(predictions, axis=1) #로짓에서 가장 높은 값을 가진 인덱스 선택

In [15]:
from sklearn.metrics import accuracy_score, classification_report

true_labels=encoded_test_sample["label"]
#예측결과와 실제 레이블을 비교해 정확도 계산
accuracy=accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

#분류 보고서 출력
report=classification_report(true_labels, predictions)
print("Classification Report:")
print(report)

Accuracy: 0.6680
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.75      0.70       254
           1       0.69      0.58      0.63       246

    accuracy                           0.67       500
   macro avg       0.67      0.67      0.66       500
weighted avg       0.67      0.67      0.67       500

